In [11]:
import pandas as pd
import numpy as np

In [12]:
def load_and_prepare_data(file_path: str) -> pd.DataFrame:
    """
    Loads data from a CSV file and performs initial preparation.
    """
    try:
        df = pd.read_csv(file_path)
        print(f"✅ Successfully loaded '{file_path}'")
    except FileNotFoundError:
        print(f"❌ Error: The file '{file_path}' was not found.")
        return pd.DataFrame()

    key_cols = ['Market', 'Submarket', 'Secondary Submarket', 'Property Type']
    for col in key_cols:
        if col in df.columns:
            df[col] = df[col].fillna('N/A')
        else:
            df[col] = 'N/A'

    df['LocationID'] = df.apply(
        lambda row: f"{row['Market']}_{row['Submarket']}_{row['Secondary Submarket']}_{row['Property Type']}",
        axis=1
    )
    df.sort_values(by=['LocationID', 'Year', 'Period_Number'], inplace=True)
    return df

In [13]:
def create_comparison_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds new columns to the DataFrame for period-over-period comparison
    without dropping any rows and with correct data types.
    """
    df['Prev_Year'] = df.groupby('LocationID')['Year'].shift(1)
    df['Prev_Period_Type'] = df.groupby('LocationID')['Period_Type'].shift(1)
    df['Prev_Period_Number'] = df.groupby('LocationID')['Period_Number'].shift(1)
    df['Prev_Inventory_SF'] = df.groupby('LocationID')['Inventory SF'].shift(1)
    df['Prev_Vacancy_Q'] = df.groupby('LocationID')['Vacancy Q'].shift(1)
    df['Prev_Asking_Rent_Q'] = df.groupby('LocationID')['Asking Rent Q'].shift(1)
    df['Prev_Net_Absorption_Q'] = df.groupby('LocationID')['Net Absorption Q'].shift(1) # Added for Net Absorption calc

    df['Prev_Year'] = df['Prev_Year'].astype('Int64')
    df['Prev_Period_Number'] = df['Prev_Period_Number'].astype('Int64')

    prev_year_float = df['Prev_Year'].astype(float)
    prev_period_float = df['Prev_Period_Number'].astype(float)

    current_abs_q = np.where(df['Period_Type'] == 'H', df['Period_Number'] * 2, df['Period_Number']) + (df['Year'] - 2000) * 4
    prev_abs_q = np.where(df['Prev_Period_Type'] == 'H', prev_period_float * 2, prev_period_float) + (prev_year_float - 2000) * 4
    
    df['Quarter_Gap'] = (current_abs_q - prev_abs_q).astype('Int64')
    
    print("✅ Successfully created comparison columns.")
    return df

In [14]:
def add_period_change_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates period-over-period changes for key metrics and adds them as new columns.
    """
    # Calculate Percentage Change, replacing division by zero or NaN results with 0
    df['Rent_Pct_Change'] = (((df['Asking Rent Q'] - df['Prev_Asking_Rent_Q']) / df['Prev_Asking_Rent_Q']).round(2)) * 100
    df['Inventory_Pct_Change'] = (((df['Inventory SF'] - df['Prev_Inventory_SF']) / df['Prev_Inventory_SF']).round(2)) * 100
    df['Vacancy_Pct_Change'] = (((df['Vacancy Q'] - df['Prev_Vacancy_Q']) / df['Prev_Vacancy_Q']).round(2)) * 100
    
    # Calculate the simple difference for Net Absorption
    df['Net_Absorption_Change'] = ((df['Net Absorption Q'] - df['Prev_Net_Absorption_Q']).round(2)) * 100
    
    print("✅ Successfully added period-over-period change columns.")
    return df

In [15]:
def run_analysis_pipeline(file_path: str) -> pd.DataFrame:
    """
    Main orchestrator that runs the simplified analysis pipeline.
    """
    # Step 1: Load and prepare the data
    prepared_df = load_and_prepare_data(file_path)
    if prepared_df.empty:
        return pd.DataFrame()

    # Step 2: Create the 'Prev_*' and 'Quarter_Gap' columns
    comparison_df = create_comparison_columns(prepared_df)
    
    # Step 3: Calculate and add the final change columns
    final_df = add_period_change_columns(comparison_df)
    
    print("\n🎉 --- Analysis Complete! ---")
    return final_df

In [16]:
file = r"csv_data/combined.csv"


In [17]:
data = run_analysis_pipeline(file)

✅ Successfully loaded 'csv_data/combined.csv'
✅ Successfully created comparison columns.
✅ Successfully added period-over-period change columns.

🎉 --- Analysis Complete! ---


In [18]:
data

,Broker,Year,Period_Type,Period_Number,Market,Submarket,Secondary Submarket,Property Type,Inventory SF,Vacancy Q,...,Prev_Period_Number,Prev_Inventory_SF,Prev_Vacancy_Q,Prev_Asking_Rent_Q,Prev_Net_Absorption_Q,Quarter_Gap,Rent_Pct_Change,Inventory_Pct_Change,Vacancy_Pct_Change,Net_Absorption_Change
701,CBRE,2023,Q,2,Albuquerque,Airport,N/A,Industrial,1111668.0,0.017,...,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2112,CBRE,2023,Q,4,Albuquerque,Airport,N/A,Industrial,1111668.0,0.094,...,2,1111668.0,0.017,11.50,-7168.0,2,-4.0,0.0,453.0,-8533200.0
2845,CBRE,2024,Q,1,Albuquerque,Airport,N/A,Industrial,1113418.0,0.068,...,4,1111668.0,0.094,11.00,-92500.0,1,3.0,0.0,-28.0,12100000.0
4226,CBRE,2024,Q,3,Albuquerque,Airport,N/A,Industrial,1113585.0,0.067,...,1,1113418.0,0.068,11.38,28500.0,2,-5.0,0.0,-1.0,-726700.0
5429,CBRE,2024,Q,4,Albuquerque,Airport,N/A,Industrial,1113585.0,0.070,...,3,1113585.0,0.067,10.81,21233.0,1,0.0,0.0,4.0,-2474100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6586,CBRE,2024,Q,4,Ventura,West County,Ventura,Industrial,9561688.0,0.023,...,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
3908,CBRE,2024,Q,1,Ventura,West Ventura,N/A,Industrial,42735424.0,0.008,...,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
5427,CBRE,2024,Q,3,Ventura,West Ventura,N/A,Total Industrial,42133571.0,0.014,...,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
3906,CBRE,2024,Q,1,Ventura,Westlake Village,N/A,Industrial,3830669.0,0.005,...,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN


In [19]:
print(data.head())

     Broker  Year Period_Type  Period_Number       Market Submarket  \
701    CBRE  2023           Q              2  Albuquerque   Airport   
2112   CBRE  2023           Q              4  Albuquerque   Airport   
2845   CBRE  2024           Q              1  Albuquerque   Airport   
4226   CBRE  2024           Q              3  Albuquerque   Airport   
5429   CBRE  2024           Q              4  Albuquerque   Airport   

     Secondary Submarket Property Type  Inventory SF  Vacancy Q  ...  \
701                  N/A    Industrial     1111668.0      0.017  ...   
2112                 N/A    Industrial     1111668.0      0.094  ...   
2845                 N/A    Industrial     1113418.0      0.068  ...   
4226                 N/A    Industrial     1113585.0      0.067  ...   
5429                 N/A    Industrial     1113585.0      0.070  ...   

      Prev_Period_Number  Prev_Inventory_SF  Prev_Vacancy_Q  \
701                 <NA>                NaN             NaN   
2112          

In [20]:
data.to_csv("data.csv",index=False)